In [19]:
#List of dependencies - Use as needed
import os
import pandas as pd
#import pycountry
#import geopandas as gpd
#import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
import requests 
import time 
import json
from pprint import pprint 
import matplotlib.pyplot as plt
import requests
from scipy import stats
import pandas as pd
from api import api_key


 
               

                #Refer to week 5 notes
        #%matplotlib widget - Allows for multiple instances of same plot
        #import matplotlib.pyplot as plt - Create plots
        #import numpy as np - mathematical manipulations
        #from scipy import stats - stats and probability

                #Refer to week 4 notes
        #from pathlib import Path - Link to csv-files
        #import pandas as pd - Read csv files

In [58]:
file_path = "Wheat.xlsx"
df1 = pd.read_excel(file_path, sheet_name='Table 15')
df1
       



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,"Table 15 Supply and disposal of wheat, canola ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Crop,1989–90,1990–91,1991–92,1992–93,1993–94,1994–95,1995–96,1996–97,...,2013–14,2014–15,2015–16,2016–17,2017–18,2018–19,2019–20,2020–21,2021–22,2022–23 s
7,NaN,NaN,kt,kt,kt,kt,kt,kt,kt,kt,...,kt,kt,kt,kt,kt,kt,kt,kt,kt,kt
8,NaN,Wheat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Production,14214.6,15066.1,10557.3,14738.7,16479.2,8961.4,16503.9,22924.2,...,25302.707,23742.56,22274.514,31818.744,20941.133,17597.561,14480.217,31922.555,36237.477,40545.42


In [56]:
file_path_two = "Wheat.xlsx"
df2 = pd.read_excel(file_path, sheet_name='Table 13')
df2

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,2021–22,61.597,685.448,0.129,1.68,0.623,3.317,0,0,0,0,0,0
71,NaN,2022–23 s,51,515,0.107,0.547,0.56,2.985,0,0,0,0,0,0
72,NaN,2023-24 f,70,693.109,0.147,0.881,0.769,5.37,0,0,0,0,0,0
73,NaN,Five–year average to 2022–23,33.5,344.918,0.119,0.791,0.657,3.817,0,0,0,0,0,0


In [ ]:
file_path_three = "Wheat_StateByState.xlsx"
df3 = pd.read_excel(file_path_three)
df3
#df1.dropna()

,Winter Crop,New South Wales,NSW,Victoria,VIC,Queensland,Q,South Australia,Unnamed: 8,Western Australia,Unnamed: 10,Tasmania,Unnamed: 12
0,NaN,area,prod.,area,prod.,area,prod.,area,prod.,area,prod.,area,prod.
1,NaN,'000 ha,kt,'000 ha,kt,'000 ha,kt,'000 ha,kt,'000 ha,kt,'000 ha,kt
2,Wheat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017–18,2 793.5,4 702.8,1 446.8,3 682.1,639.2,765.4,1 975.8,4 052.0,4 056.6,7 698.6,7.4,40.3
4,2018–19,2 382.0,1 850.3,1 402.9,2 276.6,419.7,419.7,1 820.8,3 024.5,4 369.2,9 979.2,7.6,40.1
5,2019–20,2 132.0,1 772.4,1 428.9,3 714.3,441.5,418.5,1 720.9,2 689.1,4 132.7,5 842.2,7.2,43.8
6,2020–21,4 037.0,12 972.2,1 413.4,4 525.0,966.1,1 594.2,1 831.0,3 982.2,4 387.5,8 792.7,8.2,56.3
7,2021–22,3 607.7,12 029.4,1 444.6,4 246.4,940.6,2 222.4,2 008.1,4 750.1,4 718.0,12 919.1,9.3,70
8,2022–23 s,3 600.0,10 620.0,1 500.0,5 392.9,980,2 600.0,2 200.0,7 350.0,4 750.0,14 500.0,15,82.5
9,2023-24 f,3 300.0,6 600.0,1 540.4,4 618.9,800,1 100.0,2 200.0,4 900.0,4 650.0,8 150.0,12,85


In [ ]:
df3.dropna()


,Winter Crop,New South Wales,NSW,Victoria,VIC,Queensland,Q,South Australia,Unnamed: 8,Western Australia,Unnamed: 10,Tasmania,Unnamed: 12
3,2017–18,2 793.5,4 702.8,1 446.8,3 682.1,639.2,765.4,1 975.8,4 052.0,4 056.6,7 698.6,7.4,40.3
4,2018–19,2 382.0,1 850.3,1 402.9,2 276.6,419.7,419.7,1 820.8,3 024.5,4 369.2,9 979.2,7.6,40.1
5,2019–20,2 132.0,1 772.4,1 428.9,3 714.3,441.5,418.5,1 720.9,2 689.1,4 132.7,5 842.2,7.2,43.8
6,2020–21,4 037.0,12 972.2,1 413.4,4 525.0,966.1,1 594.2,1 831.0,3 982.2,4 387.5,8 792.7,8.2,56.3
7,2021–22,3 607.7,12 029.4,1 444.6,4 246.4,940.6,2 222.4,2 008.1,4 750.1,4 718.0,12 919.1,9.3,70
8,2022–23 s,3 600.0,10 620.0,1 500.0,5 392.9,980,2 600.0,2 200.0,7 350.0,4 750.0,14 500.0,15,82.5
9,2023-24 f,3 300.0,6 600.0,1 540.4,4 618.9,800,1 100.0,2 200.0,4 900.0,4 650.0,8 150.0,12,85
10,Five–year average to 2022–23,3 151.7,7 848.9,1 437.9,4 031.1,749.6,1 450.9,1 916.2,4 359.2,4 471.5,10 406.7,9.5,58.5


In [23]:
# Save config information.

url = "http://api.openweathermap.org/data/2.5/weather?"








In [7]:

query_url = "http://api.openweathermap.org/geo/1.0/direct?q=Wangaratta,\
Victoria,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35"
print(query_url)

http://api.openweathermap.org/geo/1.0/direct?q=Wangaratta,Victoria,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35


In [8]:
response = requests.get(query_url).json()
pprint(response)

[{'country': 'AU',
  'lat': -36.356434,
  'local_names': {'en': 'Wangaratta',
                  'ja': 'ウォンガラッタ',
                  'ru': 'Уонгаратта',
                  'uk': 'Вангаратта',
                  'zh': '旺加拉塔'},
  'lon': 146.3228311,
  'name': 'Wangaratta',
  'state': 'Victoria'}]


In [21]:
query_url = "http://api.openweathermap.org/geo/1.0/direct?q=Dubbo,\
New South Whales,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35"
print(query_url)

http://api.openweathermap.org/geo/1.0/direct?q=Dubbo,New South Whales,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35


In [22]:
response = requests.get(query_url).json()
pprint(response)

[]


In [15]:
query_url = "http://api.openweathermap.org/geo/1.0/direct?q=Cairns,\
Queensland,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35"
print(query_url)

http://api.openweathermap.org/geo/1.0/direct?q=Cairns,Queensland,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35


In [16]:
response = requests.get(query_url).json()
pprint(response)

[{'country': 'AU',
  'lat': -16.9206657,
  'local_names': {'en': 'Cairns',
                  'ja': 'ケアンズ',
                  'ru': 'Кэрнс',
                  'uk': 'Кернс',
                  'zh': '凯恩斯'},
  'lon': 145.7721854,
  'name': 'Cairns',
  'state': 'Queensland'},
 {'country': 'AU',
  'lat': -17.1022281,
  'local_names': {'en': 'Cairns Regional'},
  'lon': 145.7650465401662,
  'name': 'Cairns Regional',
  'state': 'Queensland'},
 {'country': 'AU',
  'lat': -26.2251498,
  'lon': 145.5666158,
  'name': 'Cairns',
  'state': 'Queensland'}]


In [17]:
query_url = "http://api.openweathermap.org/geo/1.0/direct?q=Coffin Bay,\
South Australia,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35"
print(query_url)

http://api.openweathermap.org/geo/1.0/direct?q=Coffin Bay,South Australia,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35


In [18]:
response = requests.get(query_url).json()
pprint(response)

[{'country': 'AU',
  'lat': -34.6211018,
  'local_names': {'en': 'Coffin Bay', 'ja': 'コフィン・ベイ'},
  'lon': 135.4632303,
  'name': 'Coffin Bay',
  'state': 'South Australia'}]


In [13]:
query_url = "http://api.openweathermap.org/geo/1.0/direct?q=Northam,\
Western Australia,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35"
print(query_url)

http://api.openweathermap.org/geo/1.0/direct?q=Northam,Western Australia,Australia&limit=5&appid=ecfc1269575767c464c5c0ae6a697a35


In [14]:
response = requests.get(query_url).json()
pprint(response)

[{'country': 'AU',
  'lat': -31.6535192,
  'local_names': {'en': 'Northam', 'ru': 'Нортем'},
  'lon': 116.6726927,
  'name': 'Northam',
  'state': 'Western Australia'},
 {'country': 'AU',
  'lat': -31.699163,
  'local_names': {'en': 'Shire Of Northam'},
  'lon': 116.66485065097014,
  'name': 'Shire Of Northam',
  'state': 'Western Australia'}]


In [28]:
query_url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?\
lat=-36.356434&lon=146.3228311&dt=1706177416&appid=ecfc1269575767c464c5c0ae6a697a35"
print(query_url)

https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=-36.356434&lon=146.3228311&dt=1706177416&appid=ecfc1269575767c464c5c0ae6a697a35


In [29]:
response = requests.get(query_url).json()
pprint(response)


{'cod': 401,
 'message': 'Please note that using One Call 3.0 requires a separate '
            'subscription to the One Call by Call plan. Learn more here '
            'https://openweathermap.org/price. If you have a valid '
            'subscription to the One Call by Call plan, but still receive this '
            'error, then please see https://openweathermap.org/faq#error401 '
            'for more info.'}


In [93]:
#Dubbo Temp

csvpath = os.path.join('.','Resources', 'Dubbo_NSW_BOM', 'Dubbo_Anuual_Mean_Max_Temp.csv')
df_1 = pd.read_csv(csvpath)
print(csvpath)


# Open the CSV file and read the first 5 rows
#with open(csvpath, 'r', newline='') as file:
    #reader = csv.reader(file)
    
    # Use a loop to iterate over the first 5 rows
    #for i, row in enumerate(reader):
        #if i < 5:
            #print(row)
        #else:
            #break

# import pandas as pd
# Specify the columns you want to read
df_1= df_1[['Year','Annual']]

# Read only the specified columns from the CSV file
#df = pd.read_csv('Dubbo_Anuual_Mean_Max_Temp.csv', usecols=columns_to_read)

# Display the DataFrame
df_1.head()


.\Resources\Dubbo_NSW_BOM\Dubbo_Anuual_Mean_Max_Temp.csv


,Year,Annual
0,1993,23.4
1,1994,24.3
2,1995,23.6
3,1996,23.3
4,1997,25.0


In [94]:
#Dubbo Rain
csvpath = os.path.join('.','Resources', 'Dubbo_NSW_BOM', 'Dubbo_Annual_Rainfall.csv')
df_2 = pd.read_csv(csvpath)
print(csvpath)

df_2 = df_2[['Year','Annual']]

df_2.head()





.\Resources\Dubbo_NSW_BOM\Dubbo_Annual_Rainfall.csv


,Year,Annual
0,1994,NaN
1,1995,NaN
2,1996,NaN
3,1997,NaN
4,1998,851.0


In [95]:
#Cairns Temp
csvpath = os.path.join('.','Resources', 'Cairns_QL_BOM', 'Cairns_Annual_Mean_Max_Temp.csv')
df_3= pd.read_csv(csvpath)
print(csvpath)

df_3= df_3[['Year','Annual']]

df_3.head()





.\Resources\Cairns_QL_BOM\Cairns_Annual_Mean_Max_Temp.csv


,Year,Annual
0,2013,NaN
1,2014,29.4
2,2015,29.5
3,2016,30.1
4,2017,29.7


In [96]:
#Cairns Rain
csvpath = os.path.join('.','Resources', 'Cairns_QL_BOM', 'Cairns_Annual_Rainfall.csv')
df_4 = pd.read_csv(csvpath)
print(csvpath)

df_4 = df_4[['Year','Annual']]

df_4.head()



.\Resources\Cairns_QL_BOM\Cairns_Annual_Rainfall.csv


,Year,Annual
0,2013,NaN
1,2014,1741.0
2,2015,NaN
3,2016,1210.0
4,2017,1757.6


In [97]:
#Wangaratta Temp
csvpath = os.path.join('.','Resources', 'Wangaratta_VIC_BOM', 'Wangaratta_Annual_Mean_Max_Temp.csv')
df_5= pd.read_csv(csvpath)
print(csvpath)

df_5 = df_5[['Year','Annual']]

df_5.head()




.\Resources\Wangaratta_VIC_BOM\Wangaratta_Annual_Mean_Max_Temp.csv


,Year,Annual
0,1987,NaN
1,1988,22.2
2,1989,21.4
3,1990,21.9
4,1991,22.3


In [99]:
#Wangaratta Rain
csvpath = os.path.join('.','Resources', 'Wangaratta_VIC_BOM', 'Wangaratta_Annual_Rain.csv')
df_6 = pd.read_csv(csvpath)
print(csvpath)

df_6= df_6[['Year','Annual']]

df_6.head()




.\Resources\Wangaratta_VIC_BOM\Wangaratta_Annual_Rain.csv


,Year,Annual
0,1987,NaN
1,1988,783.0
2,1989,806.6
3,1990,674.1
4,1991,612.9


In [100]:
#Coffin Bay SA Temp Note: Wanilla closest location found
csvpath = os.path.join('.','Resources', 'CoffinBay_SA_BOM', 'Wanilla_Annual_Mean_Max_Temp.csv')

df_7 = pd.read_csv(csvpath)
print(csvpath)

df_7 = df_7[['Year','Annual']]

df_7.head()



.\Resources\CoffinBay_SA_BOM\Wanilla_Annual_Mean_Max_Temp.csv


,Year,Annual
0,1992,NaN
1,1993,21.9
2,1994,21.8
3,1995,20.3
4,1996,20.3


In [101]:
#Coffin Bay SA Rain Note: Wanilla closest location found
csvpath = os.path.join('.','Resources', 'CoffinBay_SA_BOM', 'Wanilla_Annual_Rainfall.csv')

df_8 = pd.read_csv(csvpath)
print(csvpath)

df_8 = df_8[['Year','Annual']]

df_8.head()



.\Resources\CoffinBay_SA_BOM\Wanilla_Annual_Rainfall.csv


,Year,Annual
0,1994,NaN
1,1995,379.8
2,1996,364.0
3,1997,349.2
4,1998,376.2


In [102]:
#Northam WA Temp
csvpath = os.path.join('.','Resources', 'Northam_WA_BOM', 'Northam_Annual_Mean_Max_Temp.csv')

df_9 = pd.read_csv(csvpath)
print(csvpath)

df_9 = df_9[['Year','Annual']]

df_9.head()



.\Resources\Northam_WA_BOM\Northam_Annual_Mean_Max_Temp.csv


,Year,Annual
0,1902,25.5
1,1903,24.5
2,1904,24.9
3,1905,24.7
4,1906,25.5


In [103]:
#Northam WA Rain
csvpath = os.path.join('.','Resources', 'Northam_WA_BOM', 'Northam_Annual_Rainfall.csv')
df_10 = pd.read_csv(csvpath)
print(csvpath)

df_10 = df_10[['Year','Annual']]

df_10.head()




.\Resources\Northam_WA_BOM\Northam_Annual_Rainfall.csv


,Year,Annual
0,1877,353.1
1,1878,363.2
2,1879,210.9
3,1884,431.1
4,1885,490.3


In [130]:
# Merge DataFrames based on the 'Key' column
Initial_merge = pd.merge(df_1, df_2, how="left", on=['Year'])
Initial_merge.head()


,Year,Annual_x,Annual_y
0,1993,23.4,NaN
1,1994,24.3,NaN
2,1995,23.6,NaN
3,1996,23.3,NaN
4,1997,25.0,NaN


In [134]:

Initial_merge = Initial_merge.rename(columns= {"Annual_x":"NSW Annual Max Mean Temp",
                                                "Annual_y":"NSW Annual Rainfall"})
Initial_merge.head()



,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall
0,1993,23.4,NaN
1,1994,24.3,NaN
2,1995,23.6,NaN
3,1996,23.3,NaN
4,1997,25.0,NaN


In [135]:
Second_merge = pd.merge(Initial_merge, df_3, how="left", on=['Year'])
Second_merge.head()


,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,Annual
0,1993,23.4,NaN,NaN
1,1994,24.3,NaN,NaN
2,1995,23.6,NaN,NaN
3,1996,23.3,NaN,NaN
4,1997,25.0,NaN,NaN


In [136]:

Second_merge = Second_merge.rename(columns= {"Annual":"QL Annual Max Mean Temp"})
Second_merge.head()

,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,QL Annual Max Mean Temp
0,1993,23.4,NaN,NaN
1,1994,24.3,NaN,NaN
2,1995,23.6,NaN,NaN
3,1996,23.3,NaN,NaN
4,1997,25.0,NaN,NaN


In [138]:
Third_merge = pd.merge(Second_merge, df_4, how="left", on=['Year'])


Third_merge = Third_merge.rename(columns= {"Annual":"QL Annual Rainfall"})
Third_merge.head()


,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,QL Annual Max Mean Temp,QL Annual Rainfall
0,1993,23.4,NaN,NaN,NaN
1,1994,24.3,NaN,NaN,NaN
2,1995,23.6,NaN,NaN,NaN
3,1996,23.3,NaN,NaN,NaN
4,1997,25.0,NaN,NaN,NaN


In [139]:
Fourth_merge = pd.merge(Third_merge, df_5, how="left", on=['Year'])


Fourth_merge = Fourth_merge.rename(columns= {"Annual":"VIC Annual Max Mean Temp"})
Fourth_merge.head()

,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,QL Annual Max Mean Temp,QL Annual Rainfall,VIC Annual Max Mean Temp
0,1993,23.4,NaN,NaN,NaN,21.1
1,1994,24.3,NaN,NaN,NaN,21.8
2,1995,23.6,NaN,NaN,NaN,20.7
3,1996,23.3,NaN,NaN,NaN,20.6
4,1997,25.0,NaN,NaN,NaN,22.8


In [140]:
Fifth_merge = pd.merge(Fourth_merge, df_6, how="left", on=['Year'])


Fifth_merge = Fifth_merge.rename(columns= {"Annual":"VIC Annual Rainfall"})
Fifth_merge.head()

,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,QL Annual Max Mean Temp,QL Annual Rainfall,VIC Annual Max Mean Temp,VIC Annual Rainfall
0,1993,23.4,NaN,NaN,NaN,21.1,856.8
1,1994,24.3,NaN,NaN,NaN,21.8,485.5
2,1995,23.6,NaN,NaN,NaN,20.7,824.9
3,1996,23.3,NaN,NaN,NaN,20.6,781.1
4,1997,25.0,NaN,NaN,NaN,22.8,385.0


In [141]:
Sixth_merge = pd.merge(Fifth_merge, df_7, how="left", on=['Year'])


Sixth_merge = Sixth_merge.rename(columns= {"Annual":"SA Annual Max Mean Temp"})
Sixth_merge.head()

,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,QL Annual Max Mean Temp,QL Annual Rainfall,VIC Annual Max Mean Temp,VIC Annual Rainfall,SA Annual Max Mean Temp
0,1993,23.4,NaN,NaN,NaN,21.1,856.8,21.9
1,1994,24.3,NaN,NaN,NaN,21.8,485.5,21.8
2,1995,23.6,NaN,NaN,NaN,20.7,824.9,20.3
3,1996,23.3,NaN,NaN,NaN,20.6,781.1,20.3
4,1997,25.0,NaN,NaN,NaN,22.8,385.0,20.6


In [147]:
Seventh_merge = pd.merge(Sixth_merge, df_8, how="left", on=['Year'])


Seventh_merge = Seventh_merge.rename(columns= {"Annual":"SA Annual Rainfall"})
Seventh_merge.head()

,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,QL Annual Max Mean Temp,QL Annual Rainfall,VIC Annual Max Mean Temp,VIC Annual Rainfall,SA Annual Max Mean Temp,SA Annual Rainfall
0,1993,23.4,NaN,NaN,NaN,21.1,856.8,21.9,NaN
1,1994,24.3,NaN,NaN,NaN,21.8,485.5,21.8,NaN
2,1995,23.6,NaN,NaN,NaN,20.7,824.9,20.3,379.8
3,1996,23.3,NaN,NaN,NaN,20.6,781.1,20.3,364.0
4,1997,25.0,NaN,NaN,NaN,22.8,385.0,20.6,349.2


In [148]:
Eight_merge = pd.merge(Seventh_merge, df_9, how="left", on=['Year'])


Eight_merge = Eight_merge.rename(columns= {"Annual":"WA Annual Max Mean Temp"})
Eight_merge.head()

,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,QL Annual Max Mean Temp,QL Annual Rainfall,VIC Annual Max Mean Temp,VIC Annual Rainfall,SA Annual Max Mean Temp,SA Annual Rainfall,WA Annual Max Mean Temp
0,1993,23.4,NaN,NaN,NaN,21.1,856.8,21.9,NaN,25.4
1,1994,24.3,NaN,NaN,NaN,21.8,485.5,21.8,NaN,27.0
2,1995,23.6,NaN,NaN,NaN,20.7,824.9,20.3,379.8,25.6
3,1996,23.3,NaN,NaN,NaN,20.6,781.1,20.3,364.0,25.6
4,1997,25.0,NaN,NaN,NaN,22.8,385.0,20.6,349.2,25.4


In [149]:
Temp_and_Rain_merge = pd.merge(Eight_merge, df_10, how="left", on=['Year'])


Temp_and_Rain_merge= Temp_and_Rain_merge.rename(columns= {"Annual":" WA Annual Rainfall"})
Temp_and_Rain_merge

,Year,NSW Annual Max Mean Temp,NSW Annual Rainfall,QL Annual Max Mean Temp,QL Annual Rainfall,VIC Annual Max Mean Temp,VIC Annual Rainfall,SA Annual Max Mean Temp,SA Annual Rainfall,WA Annual Max Mean Temp,WA Annual Rainfall
0,1993,23.4,NaN,NaN,NaN,21.1,856.8,21.9,NaN,25.4,320.9
1,1994,24.3,NaN,NaN,NaN,21.8,485.5,21.8,NaN,27.0,282.2
2,1995,23.6,NaN,NaN,NaN,20.7,824.9,20.3,379.8,25.6,554.4
3,1996,23.3,NaN,NaN,NaN,20.6,781.1,20.3,364.0,25.6,478.1
4,1997,25.0,NaN,NaN,NaN,22.8,385.0,20.6,349.2,25.4,394.6
5,1998,23.7,851.0,NaN,NaN,22.1,535.8,20.9,376.2,25.6,348.2
6,1999,23.4,658.4,NaN,NaN,NaN,651.4,20.8,299.0,25.5,575.9
7,2000,23.1,809.0,NaN,NaN,21.9,565.3,21.0,447.0,25.3,393.3
8,2001,24.4,611.2,NaN,NaN,22.5,577.6,20.8,469.9,25.2,306.2
9,2002,25.5,381.8,NaN,NaN,23.1,415.2,21.2,325.8,26.0,301.6
